In [1]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import glob
from scipy import stats

# Load fiducial n(z)s

In [2]:
np.random.seed(42)

In [11]:
colors2 =['C0', 'C1', 'C2', 'C3']

Likelihood  = np.load('/project/chihway/dhayaa/DECADE/SOMPZ/Runs/20240408/Final/LnLikelihood_Fiducial.npy')
max_like    = np.min(Likelihood[..., 0], axis = 0)
good        = np.abs(Likelihood[..., 0] - max_like) < stats.chi2.ppf(stats.norm.cdf(5), 8)/2

In [3]:
min_z   = 0.01
max_z   = 5
delta_z = 0.05
zbins   = np.arange(min_z,max_z+delta_z,delta_z)
zbinsc  = zbins[:-1]+(zbins[1]-zbins[0])/2.

files = sorted(glob.glob('/project/chihway/dhayaa/DECADE/SOMPZ/Runs/20240408/Final/nz_Samp*.npy'))
Final = np.concatenate([np.load(f) for f in files])
inds  = np.random.choice(Final.shape[0], 10_000, replace = False)
WZ    = Final.copy()
Final = Final[inds]

files = sorted(glob.glob('/project/chihway/dhayaa/DECADE/SOMPZ/Runs/20240408/ZP/nz_Samp*.npy'))
ZP = np.concatenate([np.load(f) for f in files])
ZP = ZP[inds]

files = sorted(glob.glob('/project/chihway/dhayaa/DECADE/SOMPZ/Runs/20240408/ZB/nz_Samp*.npy'))
ZB = np.concatenate([np.load(f) for f in files])
ZB = ZB[inds]

# Check ramping

In [11]:
for i in [0.01, 0.05, 0.1, 0.15]:
    modder       = np.where(zbinsc[None, None, :] <= i, zbinsc[None,:]/i, 1)
    Final_mod    = Final * modder
    mean_z_SOMPZ = np.trapz(Final_mod * zbinsc[None, None, :], zbinsc, axis = -1)/np.trapz(Final_mod, zbinsc, axis = -1)
    print(f"{str(i).ljust(4)} --> SOMPZ <z>:     ", np.round(np.mean(mean_z_SOMPZ, axis = 0), 4))

0.01 --> SOMPZ <z>:      [0.3345 0.5051 0.7111 0.9092]
0.05 --> SOMPZ <z>:      [0.3345 0.5051 0.7112 0.9092]
0.1  --> SOMPZ <z>:      [0.3365 0.5058 0.7114 0.9093]
0.15 --> SOMPZ <z>:      [0.3419 0.5078 0.712  0.9096]


# Estimate the mean redshifts

In [35]:
mean_z_SOMPZ = np.trapz(Final * zbinsc[None, None, :], zbinsc, axis = -1)/np.trapz(Final, zbinsc, axis = -1)
mean_z_WZ    = np.array([np.trapz(WZ[good[:, i], i] * zbinsc[None, :], zbinsc, axis = -1)/np.trapz(WZ[good[:, i], i], zbinsc, axis = -1)
                         for i in range(4)])

/tmp/jobs/41273958/ipykernel_210326/3209583755.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mean_z_WZ    = np.array([np.trapz(WZ[good[:, i], i] * zbinsc[None, :], zbinsc, axis = -1)/np.trapz(WZ[good[:, i], i], zbinsc, axis = -1)


In [36]:
print("SOMPZ <z>:     ", np.round(np.mean(mean_z_SOMPZ, axis = 0), 4))
print("SOMPZ + WZ <z>:", np.round([np.mean(mean_z_WZ[i], axis = 0) for i in range(4)], 4))

SOMPZ <z>:      [0.3344 0.505  0.7111 0.9093]
SOMPZ + WZ <z>: [0.3275 0.5051 0.7174 0.9129]


# Now the uncertainties

In [37]:
print("SOMPZ sig(<z>):     ", np.round(np.std(mean_z_SOMPZ, axis = 0), 4))
print("SOMPZ + WZ sig(<z>):", np.round([np.std(mean_z_WZ[i], axis = 0) for i in range(4)], 4))

SOMPZ sig(<z>):      [0.0069 0.0074 0.0069 0.0078]
SOMPZ + WZ sig(<z>): [0.0067 0.0079 0.0073 0.0079]


In [57]:
print("SOMPZ sig(<z>):     ", np.round(np.std(mean_z_ZP, axis = 0), 4))
print("SOMPZ sig(<z>):     ", np.round(np.std(mean_z_ZB, axis = 0), 4))

SOMPZ sig(<z>):      [0.007  0.0074 0.0069 0.0078]
SOMPZ sig(<z>):      [0.0062 0.007  0.0066 0.0077]


#### Add on the offsets specifically from ZP and ZB

In [38]:
mean_z_ZP = np.trapz(ZP * zbinsc[None, None, :], zbinsc, axis = -1)/np.trapz(ZP, zbinsc, axis = -1)
mean_z_ZB = np.trapz(ZB * zbinsc[None, None, :], zbinsc, axis = -1)/np.trapz(ZB, zbinsc, axis = -1)

In [58]:
print("SOMPZ sig(<z>), No ZB:     ", np.round(np.std(mean_z_ZP, axis = 0), 4))
print("SOMPZ sig(<z>), No ZP:     ", np.round(np.std(mean_z_ZB, axis = 0), 4))

SOMPZ sig(<z>), No ZB:      [0.007  0.0074 0.0069 0.0078]
SOMPZ sig(<z>), No ZP:      [0.0062 0.007  0.0066 0.0077]
